In [1]:
%%capture
!pip3 install google-auth google-cloud-dialogflow-cx



## imports

In [2]:
# dfcx-scrapi
from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions

# For connectopm to g-sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

# data manipulation
import pandas as pd
import numpy as np

# general python imports
import os

## g-sheets config

In [3]:
%%capture
# service account scopes
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad

scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']


#authorize sa
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/christophertrauco/.config/secrets/cx_admin_sa.json', scope)
client = gspread.authorize(credentials)
spreadsheet_key = '18qvh0uWkA0z7S5CC3xTdvD2wkhsmnINjgxUWmuoCo7M'
sh = client.open_by_key(spreadsheet_key)
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad


## master component ss id index

In [4]:
# object attribute mapping index
bc_sad_dfcx_master = sh.worksheet('Main').acell('B11').value
google_sheet_tab_read = sh.worksheet('Main').acell('B12').value



intents_ss = sh.worksheet('Index').acell('B1').value
intent_ss_output = sh.worksheet('Index').acell('D1').value
intent_ss_analysis = sh.worksheet('Index').acell('F1').value
intents_ss_hx = sh.worksheet('Index').acell('E1').value

print(intents_ss)

1lt02Hh2B_q_sluK-sAcgqyJ8WbqfZ6qDP_EQJVHPJDo


## agent object attribute mappings


In [5]:
# object data attribute mappings 
# master workbook @ https://itrau.co/bc-dfcx-sad-main

project_id = sh.worksheet('Main').acell('B2').value
agent_id = sh.worksheet('Main').acell('B13').value
display_name = sh.worksheet('Main').acell('B3').value
gcp_region = sh.worksheet('Main').acell('B4').value
zone = sh.worksheet('Main').acell('B5').value
language = sh.worksheet('Main').acell('B6').value
default_start_flow = sh.worksheet('Main').acell('B8').value
time_zone = sh.worksheet('Main').acell('B7').value
creds_path = sh.worksheet('Main').acell('B10').value
google_sheet_name = sh.worksheet('Main').acell('B11').value
google_sheet_tab_read = sh.worksheet('Main').acell('B12').value

# print(project_id, display_name, gcp_region, gcp_region, zone, language, default_start_flow,time_zone)

In [6]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = creds_path

In [7]:
parent_value = sh.worksheet('Main').acell('B13').value
print(parent_value)

projects/bot1-404120/locations/global/agents/4a2fde83-decc-4656-8995-1cbc2e3eeab4


# output selected agent intents

In [8]:
%%sh
./etl.sh

In [9]:
df = pd.read_csv('intents.csv', on_bad_lines='skip')
# print(df)

wks_name = 'Input'
cell_of_start_df = 'A1'
# upload the dataframe of the clients we want to delete
d2g.upload(df,
           intents_ss,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

APIError: {'code': 400, 'message': 'Invalid requests[0].addSheet: A sheet with the name "INPUT" already exists. Please enter another name.', 'status': 'INVALID_ARGUMENT'}

In [ ]:
# agent_id = '<YOUR_AGENT_ID_HERE>'
# google_sheet_name = 'My Google Sheet Name'
# google_sheet_tab_read = 'Read From My Tab Name'
print(agent_id)

In [ ]:
dffx = DataframeFunctions(creds_path)

df = dffx.sheets_to_dataframe(google_sheet_name, google_sheet_tab_read)
dffx.bulk_update_intents_from_dataframe(agent_id, df, update_flag=True, rate_limiter=1)

## state history changes

In [10]:
hx = client.open_by_key(intents_ss)

history = sh.worksheet('Index').acell('E1').value
# print(history)

def list_worksheets():
    wks = []
    wks_list = hx.worksheets()
    
    for x in wks_list:
       wks.append(x)
        
        
    for history in wks_list:
        if history in wks_list:
            print('History worksheet exists...')
            return
        else:
            print('History worksheet does NOT exist... Making it now. ')
            wks = hx.add_worksheet(title="History", rows=1000, cols=30)
#     log_message('Checking worksheets...')
    return wks_list
        

In [ ]:
list_worksheets()